In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} 3D visualization of Riemann sheets
TR-999
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=4) of the PDG.
:::
::::

# Riemann sheets

<!-- cspell:disable -->

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.17.0 sympy==1.12

:::{note}
In the following example we assume zero angular momentum ($L=0$).
:::

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math

warnings.filterwarnings("ignore")

## Square root definition

We define a special expression class for a square root where you can choose the sign for negative arguments. This can be used later for the phase space factor&nbsp;$\rho$.

In [ ]:
@unevaluated(real=False)
class SignedSqrt(sp.Expr):
    z: Any
    _latex_repr_ = R"\sqrt[+]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)


@unevaluated
class PosArg(sp.Expr):
    z: Any
    _latex_repr_ = R"\arg^+\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg + 2 * sp.pi, arg < -2 * sp.pi),
            (arg - 2 * sp.pi, arg > 0),
            (arg, True),
        )


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

## Phase space factor definitions

### Standard phase space

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^+\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m_a, m_b = sp.symbols("s m_a m_b")
rho_expr = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

### Chew-Mandelstamm function  

In [ ]:
@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m_a: Any
    m_b: Any
    _latex_repr_ = R"q^+\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (s * 4))


breakup_momentum = BreakupMomentum(s, m_a, m_b)
Math(aslatex({breakup_momentum: breakup_momentum.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class ChewMandelstamm(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\text{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_break = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * (sp.pi) ** 2)
            * (
                (2 * q_break / sp.sqrt(s))
                * sp.log(
                    (m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q_break) / (2 * m1 * m2)
                )
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


cm_expr = ChewMandelstamm(s, m_a, m_b)
Math(aslatex({cm_expr: cm_expr.doit(deep=False)}))

### Alternative definition of Chew-Mandelstamm

By J. L. Basdevant and Edmond L. Berger
(Phys. Rev. D 16, 657 – Published 1 August 1977)

In [ ]:
@unevaluated(real=False)
class ChewMandelstammBB(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\text{{CM}}_\text{{BB}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        rho = PhaseSpaceFactor(s, m1, m2)
        return (
            1 / sp.pi
            + (m1**2 - m2**2) / (sp.pi * s) * sp.log(m2 / m1)
            - (m1**2 + m2**2) / (sp.pi * (m1**2 - m2**2)) * sp.log(m2 / m1)
            + rho
            / sp.pi
            * sp.log(
                (sp.sqrt((m1 + m2) ** 2 - s) - sp.sqrt((m1 - m2) ** 2 - s))
                / (sp.sqrt((m1 + m2) ** 2 - s) + sp.sqrt((m1 - m2) ** 2 - s))
            )
        )


cm_bb_expr = ChewMandelstammBB(s, m_a, m_b)
Math(aslatex({cm_bb_expr: cm_bb_expr.doit(deep=False)}))

### Visual comparison

In [ ]:
args = (s, m_a, m_b)
cm_func = sp.lambdify(args, cm_expr.doit())
cm_bb_func = sp.lambdify(args, cm_bb_expr.doit())
rho_func = sp.lambdify(args, rho_expr.doit())

s_values = np.linspace(0, 2, 400)
m1_val = 0.6
m2_val = 0.4
arg_vals = (m1_val, m2_val)

cm_values = 16 * np.pi * cm_func(s_values, *arg_vals)
cm_bb_values = cm_bb_func(s_values, *arg_vals)
rho_values = 1j * rho_func(s_values, *arg_vals)

fig, axes = plt.subplots(figsize=(15, 6), ncols=3, sharex=True, sharey=True)
ax1, ax2, ax3 = axes
ax1.plot(s_values, cm_values.real, label="Real part")
ax1.plot(s_values, cm_values.imag, label="Imaginary part")
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function value")
ax1.set_title("Chew Mandelstamm Function")
ax1.legend()

ax2.plot(s_values, rho_values.real, label="Real part")
ax2.plot(s_values, rho_values.imag, label="Imaginary part")
ax2.set_xlabel("Re(s)")
ax2.set_title("Phase Space Factor")
ax2.legend()

ax3.plot(s_values, cm_bb_values.real, label="Real part")
ax3.plot(s_values, cm_bb_values.imag, label="Imaginary part")
ax3.set_xlabel("Re(s)")
ax3.set_title("Chew Mandelstamm Function BB")
ax3.legend()

ax2.set_ylim(-1.2, +1.6)
plt.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

No Blatt-Weisskopf-Formfactors because $L=0$:

In [ ]:
gamma_R, m_R, w_R = sp.symbols("gamma_R m_R Gamma_R")
k_expr = (gamma_R**2 * m_R * w_R) / (s - m_R**2)

In [ ]:
rho_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: PhaseSpaceFactor(s, m_a, m_b),  ##
}
Math(aslatex(rho_expressions))

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: (
        -sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a, m_b)
    ),  # Loop over all rho expressions
}
Math(aslatex(cm_expressions))

In [ ]:
cm_conj_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: -sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a, m_b).conjugate(),
}
Math(aslatex(cm_conj_expressions))

In [ ]:
T_rho_expr = T_explicit[0, 0].xreplace(rho_expressions)
T_cm_expr = T_explicit[0, 0].xreplace(cm_expressions)
T_cm_conj_expr = T_explicit[0, 0].xreplace(cm_conj_expressions)

In [ ]:
T_rho_expr.simplify(doit=False)

## Riemann sheet visualization

To get to the physical/unpysical Riemann sheet one uses XXXX which is derived from Schwarz reflection priciple

In [ ]:
args = (s, m_a, m_b, m_R, w_R, gamma_R)
T_rho_func = sp.lambdify(args, T_rho_expr.doit())
T_cm_func = sp.lambdify(args, T_cm_expr.doit())
T_cm_conj_func = sp.lambdify(args, T_cm_conj_expr.doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 5, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Yn = np.meshgrid(x, +y)
_, Yp = np.meshgrid(x, -y)
Zn = X + Yn * 1j
Zp = X + Yp * 1j

m1_val = 0.9
m2_val = 0.8
mR_val = 3.1
wR_val = 1.0
gamma_val = 1
arg_vals = (m1_val, m2_val, mR_val, wR_val, gamma_val)

Tp = T_rho_func(Zp**2, *arg_vals)
Tn = T_rho_func(Zn**2, *arg_vals)

Tp_cm = T_cm_func(Zp**2, *arg_vals)
Tn_cm = T_cm_func(Zn**2, *arg_vals)

Tp_cm_conj = T_cm_conj_func(Zp**2, *arg_vals)
Tn_cm_conj = T_cm_conj_func(Zn**2, *arg_vals)

In [ ]:
fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharey=True)
ax1, ax2 = axes

ax1.plot(x, Tp_cm[0].imag)
ax1.plot(x, Tn_cm_conj[0].imag)
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function Value")
ax1.set_title("Chew Mandelstamm Function")

ax2.plot(x, Tp[0].imag)
ax2.plot(x, Tn[0].imag)
ax2.set_xlabel("Re(s)")
ax2.set_title("Phsp Function")

plt.tight_layout()
plt.show()

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    n_lines = 16
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        contours=dict(
            x=dict(
                show=True,
                start=x.min(),
                end=x.max(),
                size=(x.max() - x.min()) / n_lines,
                color="black",
            ),
            y=dict(
                show=True,
                start=-y.max(),
                end=+y.max(),
                size=(y.max() - y.min()) / (n_lines // 2),
                color="black",
            ),
        ),
        name=sheet_name,
        opacity=0.6,
        showscale=False,
    )


vmax = 1.6
project = np.imag
projection_text = "Imaginary"
sheet_colors = {
    "Unphysical": "blue",
    "Physical": "red",
}

###### Sn = go.Surface(x=X, y=Yn, z=project(Tn), **sty("Unphysical"))
Sp = go.Surface(x=X, y=Yp, z=project(Tp), **sty("Physical"))
zeros = np.zeros(x.shape)

line_rho = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(Tp[0]),
    line={"color": "black", "width": 4},
    mode="lines",
    name="Lineshape",
)

threshold_point = go.Scatter3d(
    x=[(m1_val + m2_val) ** 2],
    y=[0],
    z=[0],
    mode="markers",
    marker={"color": "black", "size": 5},
    name="Threshold",
)
fig_rho = go.Figure(data=[Sn, Sp, line_rho, threshold_point])
fig_rho.update_layout(
    height=550,
    width=600,
    showlegend=False,
    title_text=(
        f"{projection_text} part of T function with standard phase space factor"
    ),
)
fig_rho.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig_rho.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Sn_cm = go.Surface(x=X, y=Yn, z=project(Tn_cm), **sty("Unphysical"))
Sp_cm = go.Surface(x=X, y=Yp, z=project(Tp_cm), **sty("Unphysical"))

Sn_cm_conj = go.Surface(x=X, y=Yn, z=project(Tn_cm_conj), **sty("Physical"))
Sp_cm_conj = go.Surface(x=X, y=Yp, z=project(Tp_cm_conj), **sty("Physical"))

intersection_points = []
for i in range(len(x)):
    intersection_points.append((x[i], 0, project(Tn_cm[0])[i]))

intersection_points = np.array(intersection_points)
intersection_line = go.Scatter3d(
    x=intersection_points[:, 0],
    y=intersection_points[:, 1],
    z=intersection_points[:, 2],
    mode="lines+markers",
    line=dict(color="yellow", width=20),
    marker=dict(size=1, color="pink"),
    name="Intersection Line",
)

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "surface"}, {"type": "surface"}, {"type": "scatter3d"}]],
)

fig.add_trace(Sn_cm, row=1, col=1)
fig.add_trace(Sp_cm, row=1, col=1)

fig.add_trace(Sn_cm_conj, row=1, col=3)
fig.add_trace(Sp_cm_conj, row=1, col=3)

fig.add_trace(Sn_cm, row=1, col=2)
fig.add_trace(Sp_cm_conj, row=1, col=2)

fig.add_trace(intersection_line, row=1, col=2)

fig.update_layout(
    height=550,
    width=1200,
    showlegend=False,
    title_text=f"{projection_text} part of T function with Chew-Mandelstam",
)
fig.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)
fig.show()

In [ ]:
%config InlineBackend.figure_formats = ['png']
import matplotlib.pyplot as plt
import numpy as np

Pn_cm = project(Tn_cm)
Pn_cm_conj = project(Tn_cm_conj)
Pp_cm = project(Tp_cm)
Pp_cm_conj = project(Tp_cm_conj)
Xgrid, Ygrid = np.meshgrid(x, zeros)

fig, axs = plt.subplots(1, 3, figsize=(15, 5))


pcm1 = axs[0].pcolormesh(X, Yn, Pn_cm, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax)
pcm1 = axs[0].pcolormesh(X, Yp, Pp_cm, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax)
pcm2 = axs[1].pcolormesh(
    X, Yn, Pn_cm_conj, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax
)
pcm2 = axs[1].pcolormesh(
    X, Yp, Pp_cm_conj, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax
)
pcm3 = axs[2].pcolormesh(X, Yp, Pp_cm, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax)
pcm3 = axs[2].pcolormesh(
    X, Yn, Pn_cm_conj, cmap=plt.cm.coolwarm, vmin=-vmax, vmax=vmax
)

fig.colorbar(pcm1, ax=axs[0], label="Unphysical")
fig.colorbar(pcm2, ax=axs[1], label="Physical")
fig.colorbar(pcm3, ax=axs[2], label="Physical and Unphysical")

axs[0].set_title("Unphysical")
axs[0].set_xlabel("Re s")
axs[0].set_ylabel("Im s")

axs[1].set_title("Physical")
axs[1].set_xlabel("Re s")
axs[1].set_ylabel("Im s")

axs[2].set_title("Physical and Unphysical combined")
axs[2].set_xlabel("Re s")
axs[2].set_ylabel("Im s")

plt.tight_layout()
plt.show()

## k-plane plots 

In [ ]:
k = sp.Symbol("k")
s_to_k_expr = sp.sqrt(((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2)) / (4 * s))
equation = sp.Eq(k, s_to_k_expr)
solutions = sp.solve(equation, s)
Math(aslatex(solutions))

In [ ]:
k_to_s_expr = solutions[0]
k_to_s_func = sp.lambdify([k, m_a, m_b], k_to_s_expr)
s_to_k_func = sp.lambdify([s, m_a, m_b], s_to_k_expr)
# s_grid = k_to_s_func(k_grid, m1_val, m2_val)

In [ ]:
s_to_k_func(Zn, m1_val, m2_val).imag.max()

In [ ]:
x_k = np.linspace(0, 20, num=200)
y_k = np.linspace(epsilon, 1, num=100)
X_k, Yn_k = np.meshgrid(x, +y)
_, Yp_k = np.meshgrid(x, -y)
Zn_k = X_k + Yn_k * 1j
Zp_k = X_k + Yp_k * 1j

In [ ]:
kn_cm = s_to_k_func(Zn_k, m1_val, m2_val)
kp_cm = s_to_k_func(Zp_k, m1_val, m2_val)

Tp_cm_k = T_cm_func(kp_cm, *arg_vals)
Tn_cm_k = T_cm_func(kn_cm, *arg_vals)

Tp_cm_conj_k = T_cm_conj_func(kp_cm, *arg_vals)
Tn_cm_conj_k = T_cm_conj_func(kn_cm, *arg_vals)


Pn_cm_k = project(Tn_cm_k)
Pn_cm_conj_k = project(Tn_cm_conj_k)
Pp_cm_k = project(Tp_cm_k)
Pp_cm_conj_k = project(Tp_cm_conj_k)
Xgrid, Ygrid = np.meshgrid(x_k, zeros)

fig_k, axs_k = plt.subplots(1, 3, figsize=(15, 5))

vmax_k = 1e-3

pcm1_k = axs_k[0].pcolormesh(
    X_k, Yn_k, Pn_cm_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)
pcm1_k = axs_k[0].pcolormesh(
    X_k, Yp_k, Pp_cm_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)
pcm2_k = axs_k[1].pcolormesh(
    X_k, Yn_k, Pn_cm_conj_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)
pcm2_k = axs_k[1].pcolormesh(
    X_k, Yp_k, Pp_cm_conj_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)
pcm3_k = axs_k[2].pcolormesh(
    X_k, Yp_k, Pp_cm_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)
pcm3_k = axs_k[2].pcolormesh(
    X_k, Yn_k, Pn_cm_conj_k, cmap=plt.cm.coolwarm, vmin=-vmax_k, vmax=vmax_k
)

fig_k.colorbar(pcm1_k, ax=axs_k[0], label="Unphysical")
fig_k.colorbar(pcm2_k, ax=axs_k[1], label="Physical")
fig_k.colorbar(pcm3_k, ax=axs_k[2], label="Physical and Unphysical")

axs_k[0].set_title("Unphysical")
axs_k[0].set_xlabel("Re s")
axs_k[0].set_ylabel("Im s")

axs_k[1].set_title("Physical")
axs_k[1].set_xlabel("Re s")
axs_k[1].set_ylabel("Im s")

axs_k[2].set_title("Physical and Unphysical combined")
axs_k[2].set_xlabel("Re s")
axs_k[2].set_ylabel("Im s")

plt.tight_layout()
plt.show()

In [ ]:
Tp_cm_conj_k = T_cm_conj_func(kp_cm, *arg_vals).real.max()